1. Data

In this homework, you will work on a dataset that contains information about a group of papers and their citation relationships.

Graphs setup
Based on the available data, you will create two graphs to model our relationships as follows:

Citation graph: This graph should represent the paper's citation relationships. We want this graph to be unweighted and directed. The citation should represent the citation given from one paper to another. For example, if paper A has cited paper B, we should expect an edge from node A to B.

Collaboration graph: This graph should represent the collaborations of the paper's authors. This graph should be weighted and undirected. Consider an appropriate weighting scheme for your edges to make your graph weighted.

Data pre-processing
The dataset is quite large and may not fit in your memory when you try constructing your graph. So, what is the solution? You should focus your investigation on a subgraph. You can work on the most connected component in the graph. However, you must first construct and analyze the connections to identify the connected components.

As a result, you will attempt to approximate that most connected component by performing the following steps:

Identify the top 10,000 papers with the highest number of citations.

Then the nodes of your graphs would be as follows:

Citation graph: you can consider each of the papers as your nodes

Collaboration graph: the authors of these papers would be your nodes

For the edges of the two graphs, you would have the following cases:

Citation graph: only consider the citation relationship between these 10,000 papers and ignore the rest.

Collaboration graph: only consider the collaborations between the authors of these 10,000 papers and ignore the rest.

The data we are working with is very large, so we needed to find a way to work with it without loading the entire file into memory at once. We made get_objects function that reads and processes large JSON files incrementally. 

The other function we worte was the one that takes id of an article as an input and returns its number of citation.

Using these two functions, we found 10000 articles with most citations.

In [24]:
import ijson
import heapq

def get_objects(filename):
    with open(filename, 'r', encoding="utf8") as f:
        objects = ijson.items(f, 'item')
        for obj in objects:
            yield obj

#a function that calculates how many citation an input article has
def number(id):
    num=0
    try:
        num=len(id['references'])
    except:
        num=0
    return num

filename = "data.json"
objects = get_objects(filename)

#top_items is a list of 10000 articles with most citations
top_items = heapq.nlargest(10000, objects, key=number)

Extracting information relevant for making a graph: first 10000 articles with most citations and their refrences among these 10000 articles

Citation graph is a list of dictionaries. Each dictionary has two keys: id and references. Id tells us which article it is and references gives us a list of ids that it cited.

In [648]:
#citation graph
rel_art = []
#the set of first 10000 articles
ids = set(i['id'] for i in top_items)

for i in top_items:
    obj = {}
    obj['id'] = i['id']
    #we are not interested if these articles have cited an article that is not in top_articles
    obj['references'] = [j for j in i['references'] if j in ids]  
    rel_art.append(obj)

Making external .txt file needed for the command line question:

In [26]:
with open('citation_graph.txt', 'w') as f:
    f.write(f'{rel_art}')

To make a collaboration graph, we had to make one dictionary and a function.

Dictionary has author ids as keys and lists of their articles as values.

The function uses information from that dictionary to find all the authors who made an article along with an author from the input. It also gives an information about the number of articles that these authors wrote with the one from an input

In [27]:
from collections import defaultdict

# Creating a dictionary where the keys are author ids and the values are lists of their articles
articles_by_author = defaultdict(list)
#We iterate over top 10000 articles
for item in top_items:
    #we iterate over authors of these top articles
    for author in item['authors']:
        # Using the 'id' component of the author dictionary as the key
        author_id = author['id']
        articles_by_author[author_id].append(item['id'])

# Converting lists of articles to sets so we can find their intersection
for author_id in articles_by_author:
    articles_by_author[author_id] = set(articles_by_author[author_id])

#function that counts how many elements 2 sets have in common
def common(set1, set2):
    c=list()
    c.append(len(set1.intersection(set2)))
    c=c+list(set1.intersection(set2))
    return c

#function that takes an author from an input and returns dictionary with authors as keys and the number of articles they made with the author from an input as values
def collaborators(aut): 
    col = {}
    # Use the 'id' component of the input author dictionary as the key
    aut_id = aut
    for author_id, articles in articles_by_author.items():
        c = common(articles, articles_by_author[aut_id])
        if (c[0] != 0 and articles!=articles_by_author[aut_id]):
            col[author_id] = c
    return col

Collaboration graph:

In [28]:
collaboration_graph=dict()
#we iterate over author ids
for i in articles_by_author.keys():
    #author ids are keys and the output of collaborators function for these ids are values
    collaboration_graph[i]=collaborators(i)

To check the accuracy of the graph, we decided to see if its outcome looks as expected and check if the graph is undirected.

First, we checked how values of the graph look like:

In [647]:
#collaboration_graph.values()

By looking at the output printed, we randomly picked the id that had number 2 as one value and checked what id has these 2 articles in common with it:

In [30]:
collaboration_graph[2131767548]

{2096665950: [2, 2811472820, 2973204678]}

Here we saw that collaboration_graph[2131767548] gives us information that author with id 2131767548 has two articles with author with an id 2096665950. We wondered if collaboration_graph[2096665950] will provide the same information, aware that if not, our collaboration graph is wrong.

In [31]:
collaboration_graph[2096665950]

{2623494704: [1, 1510836926],
 2765995116: [1, 1510836926],
 2667035442: [1, 2148956286],
 2141747120: [1, 2148956286],
 310458393: [1, 2148956286],
 2095738856: [1, 2104309040],
 2231378115: [2, 2811472820, 2973204678],
 2663712704: [2, 2811472820, 2973204678],
 2120544271: [2, 2811472820, 2973204678],
 2131767548: [2, 2811472820, 2973204678]}

It provided the same information, which made us confident that accurate information is saved in our collaboration_graph.

# 2.1 Backend Implementation

In [36]:
import networkx as nx
import numpy as np
from scipy.stats import scoreatpercentile

"CITATION GRAPH: Unweighted and directed"
"COLLABORATION GRAPH: Weighted and undirected"

citation_graph_nx = nx.DiGraph()

for node in rel_art:
    node_id = node["id"]
    references = node["references"]
    
    citation_graph_nx.add_node(node_id)
    
    for reference in references:
        citation_graph_nx.add_edge(node_id, reference)
        
collaboration_graph_nx = nx.Graph()

for node, edges in collaboration_graph.items():
    for neighbor, weight in edges.items():
        collaboration_graph_nx.add_edge(node, neighbor, weight = weight[0])
        collaboration_graph_nx[node][neighbor]["papers"] = weight[1:] 

## Functionality 1 - Graph's features

In [18]:
def graph_features(graph, name):
    number_nodes = graph.number_of_nodes()
    number_edges = graph.number_of_edges()
    density = nx.density(graph)
    if name == "Collaboration":
        degree_distribution = nx.degree_histogram(graph)
    elif name == "Citation":
        degree_distribution_citation_in = np.histogram(list(dict(graph.in_degree()).values()))
        degree_distribution_citation_out = np.histogram(list(dict(graph.out_degree()).values()))
        degree_distribution = [degree_distribution_citation_in, degree_distribution_citation_out]
    avg_degree = np.mean(list(dict(graph.degree()).values()))
    _95thpercentile = np.percentile(list(dict(graph.degree()).values()), 95)
    hubs = [node for node, degree in dict(graph.degree()).items() if degree > _95thpercentile]
    is_dense = density >= 0.5
    return {
        "Number of nodes" : number_nodes,
        "Number of edges" : number_edges,
        "Density" : density,
        "Degree distribution" : degree_distribution,
        "Average degree of the graph" : avg_degree,
        "Graph hubs" : hubs,
        "Is dense?" : is_dense        
    }

## Functionality 2 - Nodes' contribution

In [153]:
def centrality_analysis(graph, node, name):
    betweenness_centrality = nx.betweenness_centrality(graph)[node]
    pagerank_centrality = nx.pagerank(graph)[node]
    closeness_centrality = nx.closeness_centrality(graph)[node]
    degree_centrality = nx.degree_centrality(graph)[node]

    return {
        "Graph Name": name,
        "Betweenness Centrality": betweenness_centrality,
        "PageRank Centrality": pagerank_centrality,
        "Closeness Centrality": closeness_centrality,
        "Degree Centrality": degree_centrality
    }

## Functionality 3 - Shortest ordered walk

In [520]:
def bfs(graph, start_author, end_author):
    if start_author not in graph or end_author not in graph:
        return "No path", 0
    elif len(list(graph.neighbors(start_author))) == 0:
        return "No path", 0
    
    visited_authors = set()
    queue = [(start_author, [(start_author,)], [])]
    while queue:
        actual_author, path, papers = queue.pop(0)
        if actual_author not in visited_authors:
            visited_authors.add(actual_author)
            if actual_author == end_author:
                return path, papers
            neighbors = list(graph.neighbors(actual_author))
            if len(neighbors) == 0:
                return "No path", 0
            for neigh in neighbors:
                if neigh not in visited_authors:
                    queue.append((neigh, path + [(actual_author, neigh)], papers + [graph[actual_author][neigh]["papers"][0]]))
    return "No path", 0   

In [646]:
def shortest_path_order(graph, start_author, end_author, authors, N):
    graph_degrees = dict(graph.degree())
    sorted_degrees = sorted(graph_degrees, key = graph_degrees.get, reverse = True)[:N]
    top_authors = [author for author in sorted_degrees]
    graph = graph.subgraph(top_authors)
    
    authors_list = authors.copy()
    n = len(authors)
    general_path = []
    papers_visited = []
    authors = set(authors)
    path, papers = bfs(graph, start_author, authors_list[0])
    papers_visited.append(papers)
    if path == "No path":
        return path, 0
    else:
        authors_path = set([item for tup in path[1 : len(path) - 1] for item in tup])
        if len(authors_path.intersection(authors)) != 0:
            return "Order has not been respected", 0
        else:
            general_path = general_path + path
            authors.discard(authors_list[0])
    for i in range(1, n - 1):
        path, papers = bfs(graph, authors_list[i], authors_list[i + 1])
        papers_visited.append(papers)
        if path == "No path":
            return path, 0
        else:
            authors_path = set([item for tup in path[1 : len(path) - 1] for item in tup])
        if len(authors_path.intersection(authors)) != 0:
            return "Order has not been respected", 0
        else:
            general_path = general_path + path
            authors.discard(authors_list[i])
            authors.discard(authors_list[i + 1])
    path, papers=bfs(graph, authors_list[n - 1], end_author)
    papers_visited.append(papers)
    if path == "No path":
        return path, 0
    else:
        authors_path = set([item for tup in path[1 : len(path) - 1] for item in tup])
        if len(authors_path.intersection(authors)) != 0:
            return "Order has not been respected", 0
        else:
            general_path = general_path + path
            papers_visited.append(papers)
            authors.discard(authors_list[n - 1])
    return general_path, papers_visited

## Functionality 4 - Disconnecting Graphs

In [ ]:
def disconnegting_graphs(graph, authorA, authorB, N):
    graph_degrees = dict(graph.degree())
    sorted_degrees = sorted(graph_degrees, key = graph_degrees.get, reverse = True)[:N]
    top_authors = [author for author in sorted_degrees]
    graph = graph.subgraph(top_authors)

    